# 임베딩(embedding)

- 컴퓨터는 자연어를 직접 처리할 수 없기 때문에 자연어를 숫자나 벡터 형태로 변환하는 과정이 필요
- 단어나 문장을 수차화해 벡터 공간으로 표현하는 과정을 의미
- 딥러닝의 입력값으로 많이 사용됨
- 임베딩에는 문법적인 정보가 포함되어 임베딩 품질이 좋다면 단순한 모델로도 좋은 결과를 얻을 수 있음

##  단어 임베딩

- 말뭉치에서 각각의 단어를 벡터로 변환하는 기법
- 의미와 문법적 정보를 지니고 있으며, 단어를 표현하는 방법에 따라 다양한 모델이 존재

### 원-핫 인코딩(one-hot encoding)

- 단어를 숫자 벡터로 변환하는 가장 기본적인 방법
- 요소들 중 단 하나의 값만 1이고 나머지 요솟값을 0인 인코딩을 의미
- 원-핫 인코딩으로 나온 결과를 원-핫 벡터라 하며, 전체 요소 중 단 하나의 값만 1이기 때문에 희소(sparse) 벡터라고 함

- 원-핫 인코딩의 과정
    1. 단어 사전 구축
        - 단어 사전 : 말뭉치에서 나오는 서로 다른 모든 단어의 집합
        - 말뭉치에 존재하는 모든 단어의 수가 원-핫 벡터의 차원을 결정
            - 예) 100개의 단어가 존재한다면 원-핫 벡터의 크기는 100차원
            
    2. 인덱스 부여
        - 사전 내 단어 순서대로 고유한 인덱스 번호를 부여
        - 단어의 인덱스 번호가 원-핫 인코딩에서 1의 값을 가지는 요소의 위치
        
    3. 원-핫 인코딩
        - 각 단어의 고유한 인덱스에 따라 고유한 원-핫 벡터를 구성

In [1]:
from konlpy.tag import Komoran
import numpy as np

In [2]:
komoran = Komoran()

In [3]:
text = "오늘 날씨는 구름이 많아요."

In [4]:
# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [6]:
# 단어 사전 구축 및 단어별 인덱스 부여
dics = dict()

for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
        
print(dics)

{'오늘': 0, '날씨': 1, '구름': 2}


- 추출된 단어별로 인덱스를 부여해 딕셔너리의 단어와 인덱스를 저장
- 원-핫 인코딩은 동일한 단어에 서로 다른 원-핫 벡터값을 가지면 안되기 때문에 이미 저장된 단어는 다시 사전에 저장하지 않음

In [8]:
# 원-핫 인코딩
nb_classes = len(dics) # 원-핫 벡터의 차원의 크기를 결정
print(nb_classes)

3


In [9]:
targets = list(dics.values())
print(targets)

[0, 1, 2]


In [10]:
one_hot_targets = np.eye(nb_classes)[targets] # 단위행렬을 생성
print(one_hot_targets)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [11]:
np.eye(nb_classes)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [12]:
one_hot_targets[dics["오늘"]]

array([1., 0., 0.])

- 원-핫 인코딩은 간단한 구현에 비해 좋은 성능을 가지기 때문에 많이 사용됨
- 하지만 단순히 순서에 의한 인덱스값을 기반으로 인코딩된 값이기 때문에 단어의 의미나 유사한 단어와의 관계를 담지 못함
- 단어 사전의 크기가 커짐에 따라 원-핫 벡터의 차원도 커지면서 메모리 낭비와 계산의 복잡도가 함께 커짐
- 대부분의 요소가 0의 값을 가지고 있으므로 비효율적임

#### 희소 표현과 분산 표현

- 희소 표현(sparse representation)
    - 단어가 희소 벡터로 표현되는 방식
    - 각각의 차원이 독립적인 정보를 지니고 있어 사람이 이해하기에 직관적
    - 단어 사전의 크기가 커질수록 메모리 낭비와 계산 복잡도가 커지는 단점이 있음
    - 단어 간의 연관성이 전혀 없어 의미를 담을 수 없음
    
- 분산 표현(distributed representation
    - 한 단어의 정보가 특정 차원에 표현되지 않고 여러 차원에 분산되어 표현됨
    - 하나의 차원에 다양한 정보를 가지고 있음
    - 장점
        - 임베딩 벡터의 차원 데이터 손실을 최소화하면서 압축할 수 있음
            - 적은 차원으로 많은 단어를 표현할 수 있음
        - 단어의 의미, 주변 단어간의 관계 등 많은 정보가 내표되어 있어 일반화 능력이 뛰어남
            - 예) 남자와 남성이라는 단어는 희소 표현 방식에서는 각각의 요솟값에 불과함
            - 분산 표현 방식에서는 유사한 의미를 가지는 단어들은 비슷한 위치로 분포하기 때문에 남자와 남성이라는 단어의 위치가 가까움
            - 두 단어간의 거리를 계산한다면 컴퓨터는 남자와 남성을 같은 의미로 해석할 수 있음

### Word2Vec

- 신경망 기반 단어 임베딩
- 2013년 구글에서 발표
- 기존 신경망 기반 단어 임베딩에서 계산량을 줄여 빠른 학습이 가능하게 함
    - CBOW(continuous bag-of-words)와 skip-gram 두 가지 모델로 제안됨
- CBOW
    - 주변 단어들(context word)을 이용해 타깃 단어를 예측하는 신경망 모델
    - 신경망의 입력을 주변 단어들로 구성하고 출력을 타깃 단어로 설정해 학습된 가중치 데이터를 임베딩 벡터로 활용
    - 학습 속도가 빠름
    
- skip-gram
    - 하나의 타깃 단어를 이용해 주변 단어들을 예측하는 신경망 모델
    - CBOW 모델에 비해 예측해야하는 맥락이 많아짐
    - 단어 분산 표현력이 우수해 CBOW 모델에 비해 임베딩 품질이 우수

<img src = "./image/cbow_skip-gram.png">

- 타깃 단어를 예측하기 위해 앞뒤 단어를 확인하는데 몇 개의 단어까지 확인할지 결정하는 범위를 윈도우(window)라고 함

<img src = "./image/w2v_window_size.png">

- 학습을 통해 의미상 비슷한 단어들을 비슷한 벡터 공간에 위치시킴
- 벡터의 특성상 의미에 따라 방향성을 갖게 됨
- 임베딩 된 벡터들 간연산이 가능하기 때문에 단어간 관계를 계산할 수 있음